In [1]:
import numpy as np

In [5]:
# helper functions here
 
nInst=100
currentPos = np.zeros(nInst)

# Dummy algorithm to demonstrate function format.
def getMyPosition (prcSoFar):
    global currentPos
    # (nins,nt) = prcSoFar.shape
    # rpos = np.array([int(x) for x in 1000 * np.random.randn(nins)])
    # currentPos += rpos
    # The algorithm must return a vector of integers, indicating the position of each stock.
    # Position = number of shares, and can be positve or negative depending on long/short position.
    return currentPos

In [6]:
# run eval.py code here
!python eval.py

Loaded 100 instruments for 250 days
Day 201 value: -3737.33 todayPL: $-3737.33 $-traded: 747467 return: -0.00500 frac0: 0.4997 frac1: 0.5003
Day 202 value: -11230.63 todayPL: $-7493.30 $-traded: 1148510 return: -0.00978 frac0: 0.5013 frac1: 0.4987
Day 203 value: -9774.77 todayPL: $1455.87 $-traded: 1625337 return: -0.00601 frac0: 0.5036 frac1: 0.4964
Day 204 value: -11760.43 todayPL: $-1985.66 $-traded: 1946425 return: -0.00604 frac0: 0.4968 frac1: 0.5032
Day 205 value: -15961.83 todayPL: $-4201.41 $-traded: 2227999 return: -0.00716 frac0: 0.4862 frac1: 0.5138
Day 206 value: -16411.52 todayPL: $-449.69 $-traded: 2442794 return: -0.00672 frac0: 0.4818 frac1: 0.5182
Day 207 value: -12689.28 todayPL: $3722.25 $-traded: 2694997 return: -0.00471 frac0: 0.4822 frac1: 0.5178
Day 208 value: -12556.13 todayPL: $133.15 $-traded: 2826779 return: -0.00444 frac0: 0.4783 frac1: 0.5217
Day 209 value: -11149.19 todayPL: $1406.94 $-traded: 3063623 return: -0.00364 frac0: 0.4769 frac1: 0.5231
Day 210 va